# Interactive Plots
If you want to type along with me, use [this notebook](https://humboldt.cloudbank.2i2c.cloud/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fbethanyj0%2Fdata271_sp24&branch=main&urlpath=tree%2Fdata271_sp24%2Fdemos%2Fdata271_demo37_live.ipynb) instead. 
If you don't want to type and want to follow along just by executing the cells, stay in this notebook. 

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns

### Basics with Plotly Express
**Hover capabilities**

In [ ]:
# Import plotly express
import plotly.express as px

In [ ]:
# Create a basic line plot
t = np.linspace(0, 2*np.pi, 100)
fig = px.line(x=t, y=np.cos(t))
fig.show()

In [ ]:
# Include more details in the hover feature
fig = px.line(x=t, y=np.cos(t), labels={'x':'t', 'y':'cos(t)'})
fig.show()

In [ ]:
# Simple scatter plot
df = px.data.iris() # plotly has some built in datasets (like seaborn and plotnine)

fig = px.scatter(df, x="sepal_length", y="sepal_width",color='species', width=500, height=400)
fig.show()

In [ ]:
# What other info is in the dataframe?
df

In [ ]:
# add more data to the hover feature
fig = px.scatter(df, x="sepal_length", y="sepal_width",color='species',
                 width=500, height=400,
                hover_data=['petal_length','petal_width'])
fig.show()

In [ ]:
# Facetting is also possible
fig = px.scatter(df, x="sepal_length", y="sepal_width",color='species',
                 width=1000, height=400,
                hover_data=['petal_length','petal_width'],
                facet_col='species')
fig.show()

In [ ]:
# Histograms
fig = px.histogram(df, x="sepal_length", 
                 width=500, height=400)
fig.show()

In [ ]:
# Create heatmap using Plotly Express
fig = px.imshow(
  df.corr(numeric_only=True), color_continuous_scale='plasma'
)

# Show plot
fig.show()

### Adding dropdown menus and sliders with ipywidgets

In [ ]:
from ipywidgets import interact
import ipywidgets as widgets

# Generate some data
x = np.linspace(0, 10, 100)
y1 = np.sin(x)
y2 = np.cos(x)

# Function to update the plot based on dropdown selection
def update_plot(change):
    if change == 'sin':
        plt.plot(x, y1, label='sin(x)')
    elif change == 'cos':
        plt.plot(x, y2, label='cos(x)')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.legend()
    plt.show()

# Create a dropdown widget with options
options = {'sin(x)': 'sin', 'cos(x)': 'cos'}
dropdown = widgets.Dropdown(options=options, description='Function:')

# Use interact to connect the dropdown and update_plot function
interact(update_plot, change=dropdown);

In [ ]:
# Sliders
def update_plot(slider_value):
    plt.plot(x, np.sin(x * slider_value))
    plt.xlabel('x')
    plt.ylabel('sin(f*x)')
    plt.show()

# Create a slider widget using widget.FloatSlider
slider = widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=2.0,
    step=0.1,
    description='Frequency (f):',
    continuous_update=True
)

# Use interact to connect the slider and the update_plot function
interact(update_plot, slider_value=slider);

#### Data example

In [ ]:
# Create a plot with adjustable x and y axes
# Function to update the scatter plot based on dropdown selection
def update_plot(x_axis, y_axis):
    fig = px.scatter(df, x=x_axis, y=y_axis, color='species',
                    height = 500, width=700)
    fig.show()

# Define dropdown options for x and y axes
x_options = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
y_options = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

# Create dropdown widgets
x_dropdown = widgets.Dropdown(options=x_options, description='X Axis:')
y_dropdown = widgets.Dropdown(options=y_options, description='Y Axis:')

# Create interactive plot using ipywidgets interact function
interact(update_plot, x_axis=x_dropdown, y_axis=y_dropdown);

### Animations/sliders with plotly

In [ ]:
# Import the gap minder
df = pd.read_csv('gapminder_all.csv')
df

In [ ]:
# Tidy the data (start by melting)
long_df = df.melt(id_vars=['continent','country'])
long_df

In [ ]:
# Each variable gets its own column
long_df[['var','year']] = long_df.variable.str.split('_',expand=True)
long_df.drop(columns = 'variable',inplace=True)
long_df

In [ ]:
# pivot so each variable is a column
tidy_df = long_df.pivot_table(index = ['continent','country','year'],columns='var',values='value').reset_index()
tidy_df

In [ ]:
# check info for datatypes and nulls
tidy_df.info()

In [ ]:
# cast year to type int
tidy_df['year'] = tidy_df.year.astype(int)

### How would we visualize with Seaborn?

In [ ]:
# scatterplot in seaborn for one year
sns.scatterplot(data = tidy_df[tidy_df['year']== 2007],
                x = 'gdpPercap',y='lifeExp',size='pop', 
                sizes=(25, 250), hue='continent',alpha=0.7)
plt.xscale('log')

In [ ]:
# Show the trend across years
sns.relplot(data = tidy_df,x = 'gdpPercap',y='lifeExp',
            size='pop', sizes=(25, 250), 
            hue='continent',alpha=0.7,
            col='year',col_wrap=4)
plt.xscale('log')

### Same data with interactive plots

In [ ]:
# Look at just one year
fig = px.scatter(tidy_df[tidy_df['year']==2007], x="gdpPercap", y="lifeExp",
           size="pop", color="continent", 
           log_x=True, size_max=55,
           hover_name='country')
fig.show()

In [ ]:
# Create animation/slider instead of facetting
fig = px.scatter(tidy_df, x="gdpPercap", y="lifeExp",
           size="pop", color="continent", 
           hover_name="country",
           animation_frame="year", 
           log_x=True, size_max=55, range_x=[100,100000], range_y=[25,90])
fig.show()

In [ ]:
# Combine animations with dropdowns
def update_plot(option):
    if option:
        x_scale = [100,100000]
    else:
        x_scale = [0,45000]
    fig = px.scatter(tidy_df, x="gdpPercap", y="lifeExp",
           size="pop", color="continent", 
           hover_name="country",
           animation_frame="year", 
           log_x=option, size_max=55, range_x=x_scale, range_y=[25,90])
    fig.show()

# Define dropdown options for x and y axes
scale_options = [True, False]

# Create dropdown widgets
scale_dropdown = widgets.Dropdown(options=scale_options, description='Log scale:')

# Create interactive plot using ipywidgets interact function
interact(update_plot, option=scale_dropdown);



In [ ]:
# Can also make animated bar charts
fig = px.bar(tidy_df, x="continent", y="pop", color="continent",
              animation_frame="year", hover_name = 'country',range_y=[0,4000000000])
fig.show()